# Mix'n'match Mismatch Generation

This notebook is used to genrate mismatches for [Mismatch Finder](https://www.wikidata.org/wiki/Wikidata:Mismatch_Finder) via a request to [Mix'n'match](https://meta.wikimedia.org/wiki/Mix%27n%27match) data stores. Data will be formatted for upload given the [directions for creating a mismatch file](https://github.com/wmde/wikidata-mismatch-finder/blob/main/docs/UserGuide.md#creating-a-mismatches-import-file).

In [4]:
#!pip install jupyter-black
#!pip install tensorflow
#!pip install aiohttp

In [5]:
# %load_ext jupyter_black

In [6]:
import ast
import json
import sys
import urllib

import numpy as np
import pandas as pd

PATH_TO_UTILS = "../"  # change based on your directory structure
sys.path.append(PATH_TO_UTILS)

from utils import check_mf_formatting

## Get Data

In [7]:
mnm_mismatch_request_url = (
    "https://mix-n-match.toolforge.org/api.php?query=all_issues&mode=time_mismatch"
)

In [8]:
with urllib.request.urlopen(mnm_mismatch_request_url) as url:
    mnm_mismatch_data = json.load(url)

In [9]:
mnm_mismatch_data_expanded = []
for d in mnm_mismatch_data["data"]:
    d["source"] = f"https://mix-n-match.toolforge.org/#/entry/{d['entry_id']}"
    d.pop("issue_id", None)
    d["time_mismatch"]["pid"] = d["time_mismatch"].pop("prop")
    d["time_mismatch"]["qid"] = d["time_mismatch"].pop("q")
    d["item_id"] = d["time_mismatch"]["qid"]

    mnm_mismatch_data_expanded.append(d)

## Explore data

In [10]:
print(f"{len(mnm_mismatch_data['data']):,}")

82,996


In [11]:
mnm_mismatch_data["data"][:2]

[{'entry_id': '44032422',
  'time_mismatch': {'wd_time': '+1925-01-01T00:00:00Z',
   'mnm_time': '+1926-07-04T00:00:00Z',
   'pid': 'P569',
   'qid': 'Q329124'},
  'source': 'https://mix-n-match.toolforge.org/#/entry/44032422',
  'item_id': 'Q329124'},
 {'entry_id': '115714460',
  'time_mismatch': {'wd_time': '+1998-09-19T00:00:00Z',
   'mnm_time': '+1987-04-17T00:00:00Z',
   'pid': 'P569',
   'qid': 'Q107654539'},
  'source': 'https://mix-n-match.toolforge.org/#/entry/115714460',
  'item_id': 'Q107654539'}]

In [12]:
mnm_mismatch_data_expanded[:2]

[{'entry_id': '44032422',
  'time_mismatch': {'wd_time': '+1925-01-01T00:00:00Z',
   'mnm_time': '+1926-07-04T00:00:00Z',
   'pid': 'P569',
   'qid': 'Q329124'},
  'source': 'https://mix-n-match.toolforge.org/#/entry/44032422',
  'item_id': 'Q329124'},
 {'entry_id': '115714460',
  'time_mismatch': {'wd_time': '+1998-09-19T00:00:00Z',
   'mnm_time': '+1987-04-17T00:00:00Z',
   'pid': 'P569',
   'qid': 'Q107654539'},
  'source': 'https://mix-n-match.toolforge.org/#/entry/115714460',
  'item_id': 'Q107654539'}]

In [13]:
mnm_mismatch_data_expanded = list(filter(lambda d: d["time_mismatch"]["wd_time"] != d["time_mismatch"]["mnm_time"], mnm_mismatch_data_expanded))
len(mnm_mismatch_data_expanded)

82996

## Sync implementation

In [14]:
import pandas as pd
from numpy import NAN
from tqdm import tqdm

In [ ]:
acc = []
for entry in tqdm(mnm_mismatch_data_expanded):
    data = entry["time_mismatch"]
    req = f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{entry["item_id"]}?_fields=statements'
    try:
        with urllib.request.urlopen(req) as url:
            wd_props = json.load(url)["statements"]
    except urllib.request.HTTPError as e:
        # Fixed in newer version https://stackoverflow.com/questions/67723860/python-urllib-request-urlopen-http-error-308-permanent-redirect
        print("Skipped", req)
        print(e)
        continue

    with urllib.request.urlopen(f'https://mix-n-match.toolforge.org/api.php?query=get_entry&entry={entry["entry_id"]}') as url:
        ext_url = json.load(url)["data"]["entries"][entry["entry_id"]]["ext_url"]
    
    acc.append({
        "item_id": entry["item_id"],
        "statement_guid": wd_props[data["pid"]][0]["id"],
        "property_id": data["pid"],
        "wikidata_value": data["wd_time"],
        "meta_wikidata_value": NAN,
        "external_value": data["mnm_time"],
        "external_url": ext_url,
        "type": "statement",
    })

## Async implementation
(currently too fast)

In [15]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
import aiohttp
import ssl
import json

acc = []

async def fetch(session, entry):
    data = entry["time_mismatch"]
    req = f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{entry["item_id"]}?_fields=statements'
    async with session.get(req, ssl=ssl.SSLContext()) as response:
        try:
            wd_props = (await response.json())["statements"]
        except:
            print(await response.text())
        pbar.update(0.5)

        url = f'https://mix-n-match.toolforge.org/api.php?query=get_entry&entry={entry["entry_id"]}'
        async with session.get(url, ssl=ssl.SSLContext()) as response:
            ext_url = (await response.json())["data"]["entries"][entry["entry_id"]]["ext_url"]
            pbar.update(0.5)
        
            acc.append({
                "item_id": entry["item_id"],
                "statement_guid": wd_props[data["pid"]][0]["id"],
                "property_id": data["pid"],
                "wikidata_value": data["wd_time"],
                "meta_wikidata_value": NAN,
                "external_value": data["mnm_time"],
                "external_url": ext_url,
                "type": "statement",
            })
            return True


async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:  # read_timeout=None
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results


if __name__ == '__main__':
    pbar = tqdm(total=len(mnm_mismatch_data_expanded))
    loop = asyncio.get_event_loop()
    status = loop.run_until_complete(fetch_all(mnm_mismatch_data_expanded, loop))
    pbar.close()


  0%|          | 0/82996 [00:00<?, ?it/s]/tmp/ipykernel_2014607/4271199208.py:11: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  async with session.get(req, ssl=ssl.SSLContext()) as response:
/tmp/ipykernel_2014607/4271199208.py:11: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  async with session.get(req, ssl=ssl.SSLContext()) as response:
  0%|          | 0.5/82996 [00:33<1559:19:00, 67.64s/it]/tmp/ipykernel_2014607/4271199208.py:19: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  async with session.get(url, ssl=ssl.SSLContext()) as response:
/tmp/ipykernel_2014607/4271199208.py:19: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  async with session.get(url, ssl=ssl.SSLContext()) as response:
  3%|▎         | 2687.5/82996 [03:02<1:21:39, 16.39it/s]

<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1 { margin-top: 1em; font-size: 1.2em; }
.content-text { overflow: hidden; overflow-wrap: break-word; word-wrap: break-word; -webkit-hyphens: auto; -moz-hyphens: auto; -ms-hyphens: auto; hyphens: auto; }
p { margin: 0.7em 0 1em 0; }
a { color: #0645ad; text-decoration: none; }
a:hover { text-decoration: underline; }
code { font-family: sans-serif; }
.text-muted { color: #777; }
</style>
<div class="content" role="main">
<a href="https://www.wikimedia.org"><img src="https://www.wikimedia.org/static/images/wmf-logo.png" s

  3%|▎         | 2720.0/82996 [03:04<1:01:56, 21.60it/s]

<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1 { margin-top: 1em; font-size: 1.2em; }
.content-text { overflow: hidden; overflow-wrap: break-word; word-wrap: break-word; -webkit-hyphens: auto; -moz-hyphens: auto; -ms-hyphens: auto; hyphens: auto; }
p { margin: 0.7em 0 1em 0; }
a { color: #0645ad; text-decoration: none; }
a:hover { text-decoration: underline; }
code { font-family: sans-serif; }
.text-muted { color: #777; }
</style>
<div class="content" role="main">
<a href="https://www.wikimedia.org"><img src="https://www.wikimedia.org/static/images/wmf-logo.png" s

  3%|▎         | 2725.0/82996 [03:04<1:12:10, 18.54it/s]

...

  5%|▌         | 4270.0/82996 [04:29<1:05:32, 20.02it/s]

<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1 { margin-top: 1em; font-size: 1.2em; }
.content-text { overflow: hidden; overflow-wrap: break-word; word-wrap: break-word; -webkit-hyphens: auto; -moz-hyphens: auto; -ms-hyphens: auto; hyphens: auto; }
p { margin: 0.7em 0 1em 0; }
a { color: #0645ad; text-decoration: none; }
a:hover { text-decoration: underline; }
code { font-family: sans-serif; }
.text-muted { color: #777; }
</style>
<div class="content" role="main">
<a href="https://www.wikimedia.org"><img src="https://www.wikimedia.org/static/images/wmf-logo.png" s

  5%|▌         | 4275.5/82996 [04:29<58:31, 22.42it/s]  

<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1 { margin-top: 1em; font-size: 1.2em; }
.content-text { overflow: hidden; overflow-wrap: break-word; word-wrap: break-word; -webkit-hyphens: auto; -moz-hyphens: auto; -ms-hyphens: auto; hyphens: auto; }
p { margin: 0.7em 0 1em 0; }
a { color: #0645ad; text-decoration: none; }
a:hover { text-decoration: underline; }
code { font-family: sans-serif; }
.text-muted { color: #777; }
</style>
<div class="content" role="main">
<a href="https://www.wikimedia.org"><img src="https://www.wikimedia.org/static/images/wmf-logo.png" s

  5%|▌         | 4278.0/82996 [04:29<57:48, 22.69it/s]

In [1]:
nn = list(filter(lambda x: x is not None, acc))
nn[2]

NameError: name 'acc' is not defined

## Saving

In [ ]:
mismatchDF = pd.DataFrame(acc)
mismatchDF

In [ ]:
check_mf_formatting(mismatchDF)
len(mismatchDF)

In [ ]:
mismatchDF.to_csv("first-4327-mismatches.csv")